In [1]:
import utils

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import matplotlib.pyplot as plt
from transformers import XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer
%matplotlib inline

Using TensorFlow backend.
/home/danasour/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/danasour/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/danasour/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/danasour/.local/lib/python3.6/site-packages/tensorflo

In [2]:
MODEL_CLASSES = {
     'xlnet': (          XLNetForSequenceClassification,         XLNetTokenizer,         'xlnet-base-cased')
}
model_class, tokenizer, pretrained_model = MODEL_CLASSES['xlnet']
tokenizer = tokenizer.from_pretrained(pretrained_model)

In [5]:
def create_data(csv_file):
    
    data = pd.read_csv(csv_file).fillna("")
    tr_qids = pd.Series(data['qid1'].tolist() + data['qid2'].tolist())
    unique_dic = tr_qids.value_counts()
    sentences = [None]*len(unique_dic)
    data = np.array(data)
    
    for row in data:
        for i in range(1,3):
            id = row[i]
            if not sentences[id-1]:
                question = row[i+2]

                embedding = tokenizer.encode(question) 
                if len(embedding) > 34:
                    embedding = embedding[0:34]
                else:
                    embedding.extend([0] * (34 - len(embedding)))
                
                sentences[id-1] = [question] + embedding

    df = pd.DataFrame(sentences)
    return df

In [6]:
file_OUT = "data/sentences.csv"
file_IN = "data/train.csv"

df = create_data(file_IN)
utils.WriteCSV(df, file_OUT)